In [1]:
from sympy import *

# Derivaties for the extended GAM implementation

\begin{align}
y&: y \\
a&: \alpha = \exp(\theta_{0}) \\
n&: \eta \\
g&: \gamma \\
\end{align}
where $\theta_{0}\in \mathbb{R}$ is an unrestricted parameter.

In [2]:
y, t0, n, g = symbols("y, t0, n, g")
a = exp(t0)

In [3]:
l_ZTNB = (
    y * log(a)
    + y * g
    - y * log(1 + a * exp(g))
    + log(gamma(y + 1 / a))
    + log(gamma(y + 1))
    - log(gamma(1 / a))
    - log(((1 + a * exp(g)) ** (1 / a)) - 1)
)
l_ZTNB

g*y - y*log(exp(g)*exp(t0) + 1) + y*log(exp(t0)) - log((exp(g)*exp(t0) + 1)**exp(-t0) - 1) + log(gamma(y + 1)) + log(gamma(y + exp(-t0))) - log(gamma(exp(-t0)))

In [4]:
# for y == 0
l0_ZINB = -exp(n)
# for y > 0
l_ZINB = log(1 - exp(-exp(n))) + l_ZTNB

l_ZINB

g*y - y*log(exp(g)*exp(t0) + 1) + y*log(exp(t0)) + log(1 - exp(-exp(n))) - log((exp(g)*exp(t0) + 1)**exp(-t0) - 1) + log(gamma(y + 1)) + log(gamma(y + exp(-t0))) - log(gamma(exp(-t0)))

## Derivatives

#### w.r.t. $\eta$

In [5]:
l_ZINB.diff(n, 1)

exp(n)*exp(-exp(n))/(1 - exp(-exp(n)))

In [6]:
l_ZINB.diff(n, 2)

(-exp(n) + 1 - exp(n)*exp(-exp(n))/(1 - exp(-exp(n))))*exp(n)*exp(-exp(n))/(1 - exp(-exp(n)))

In [7]:
l_ZINB.diff(n, 3)

(exp(2*n) - 3*exp(n) + 1 + 3*exp(2*n)*exp(-exp(n))/(1 - exp(-exp(n))) - 3*exp(n)*exp(-exp(n))/(1 - exp(-exp(n))) + 2*exp(2*n)*exp(-2*exp(n))/(1 - exp(-exp(n)))**2)*exp(n)*exp(-exp(n))/(1 - exp(-exp(n)))

In [8]:
l_ZINB.diff(n, 4)

(-exp(3*n) + 6*exp(2*n) - 7*exp(n) + 1 - 7*exp(3*n)*exp(-exp(n))/(1 - exp(-exp(n))) + 18*exp(2*n)*exp(-exp(n))/(1 - exp(-exp(n))) - 7*exp(n)*exp(-exp(n))/(1 - exp(-exp(n))) - 12*exp(3*n)*exp(-2*exp(n))/(1 - exp(-exp(n)))**2 + 12*exp(2*n)*exp(-2*exp(n))/(1 - exp(-exp(n)))**2 - 6*exp(3*n)*exp(-3*exp(n))/(1 - exp(-exp(n)))**3)*exp(n)*exp(-exp(n))/(1 - exp(-exp(n)))

#### w.r.t. $\gamma$

In [9]:
l_ZINB.diff(g, 1)

y - y*exp(g)*exp(t0)/(exp(g)*exp(t0) + 1) - (exp(g)*exp(t0) + 1)**exp(-t0)*exp(g)/((exp(g)*exp(t0) + 1)*((exp(g)*exp(t0) + 1)**exp(-t0) - 1))

In [10]:
l_ZINB.diff(g, 2)

(-y*exp(t0) + y*exp(g)*exp(2*t0)/(exp(g)*exp(t0) + 1) - (exp(g)*exp(t0) + 1)**exp(-t0)/((exp(g)*exp(t0) + 1)**exp(-t0) - 1) + (exp(g)*exp(t0) + 1)**(2*exp(-t0))*exp(g)/((exp(g)*exp(t0) + 1)*((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**2) + (exp(g)*exp(t0) + 1)**exp(-t0)*exp(g)*exp(t0)/((exp(g)*exp(t0) + 1)*((exp(g)*exp(t0) + 1)**exp(-t0) - 1)) - (exp(g)*exp(t0) + 1)**exp(-t0)*exp(g)/((exp(g)*exp(t0) + 1)*((exp(g)*exp(t0) + 1)**exp(-t0) - 1)))*exp(g)/(exp(g)*exp(t0) + 1)

In [11]:
l_ZINB.diff(g, 3)

(-y*exp(t0) + 3*y*exp(g)*exp(2*t0)/(exp(g)*exp(t0) + 1) - 2*y*exp(2*g)*exp(3*t0)/(exp(g)*exp(t0) + 1)**2 - (exp(g)*exp(t0) + 1)**exp(-t0)/((exp(g)*exp(t0) + 1)**exp(-t0) - 1) + 3*(exp(g)*exp(t0) + 1)**(2*exp(-t0))*exp(g)/((exp(g)*exp(t0) + 1)*((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**2) + 3*(exp(g)*exp(t0) + 1)**exp(-t0)*exp(g)*exp(t0)/((exp(g)*exp(t0) + 1)*((exp(g)*exp(t0) + 1)**exp(-t0) - 1)) - 3*(exp(g)*exp(t0) + 1)**exp(-t0)*exp(g)/((exp(g)*exp(t0) + 1)*((exp(g)*exp(t0) + 1)**exp(-t0) - 1)) - 2*(exp(g)*exp(t0) + 1)**(3*exp(-t0))*exp(2*g)/((exp(g)*exp(t0) + 1)**2*((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**3) - 3*(exp(g)*exp(t0) + 1)**(2*exp(-t0))*exp(2*g)*exp(t0)/((exp(g)*exp(t0) + 1)**2*((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**2) + 3*(exp(g)*exp(t0) + 1)**(2*exp(-t0))*exp(2*g)/((exp(g)*exp(t0) + 1)**2*((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**2) - 2*(exp(g)*exp(t0) + 1)**exp(-t0)*exp(2*g)*exp(2*t0)/((exp(g)*exp(t0) + 1)**2*((exp(g)*exp(t0) + 1)**exp(-t0) - 1)) + 3*(exp(g)*exp(t0) + 1)**exp(-t0)*ex

In [12]:
l_ZINB.diff(g, 4)

(-y*exp(t0) + 7*y*exp(g)*exp(2*t0)/(exp(g)*exp(t0) + 1) - 12*y*exp(2*g)*exp(3*t0)/(exp(g)*exp(t0) + 1)**2 + 6*y*exp(3*g)*exp(4*t0)/(exp(g)*exp(t0) + 1)**3 - (exp(g)*exp(t0) + 1)**exp(-t0)/((exp(g)*exp(t0) + 1)**exp(-t0) - 1) + 7*(exp(g)*exp(t0) + 1)**(2*exp(-t0))*exp(g)/((exp(g)*exp(t0) + 1)*((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**2) + 7*(exp(g)*exp(t0) + 1)**exp(-t0)*exp(g)*exp(t0)/((exp(g)*exp(t0) + 1)*((exp(g)*exp(t0) + 1)**exp(-t0) - 1)) - 7*(exp(g)*exp(t0) + 1)**exp(-t0)*exp(g)/((exp(g)*exp(t0) + 1)*((exp(g)*exp(t0) + 1)**exp(-t0) - 1)) - 12*(exp(g)*exp(t0) + 1)**(3*exp(-t0))*exp(2*g)/((exp(g)*exp(t0) + 1)**2*((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**3) - 18*(exp(g)*exp(t0) + 1)**(2*exp(-t0))*exp(2*g)*exp(t0)/((exp(g)*exp(t0) + 1)**2*((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**2) + 18*(exp(g)*exp(t0) + 1)**(2*exp(-t0))*exp(2*g)/((exp(g)*exp(t0) + 1)**2*((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**2) - 12*(exp(g)*exp(t0) + 1)**exp(-t0)*exp(2*g)*exp(2*t0)/((exp(g)*exp(t0) + 1)**2*((exp(g)*exp(t0) + 1)

#### w.r.t. $\theta_0$

In [13]:
l_ZINB.diff(t0, 1)

y - y*exp(g)*exp(t0)/(exp(g)*exp(t0) + 1) - (exp(g)*exp(t0) + 1)**exp(-t0)*(-exp(-t0)*log(exp(g)*exp(t0) + 1) + exp(g)/(exp(g)*exp(t0) + 1))/((exp(g)*exp(t0) + 1)**exp(-t0) - 1) - exp(-t0)*polygamma(0, y + exp(-t0)) + exp(-t0)*polygamma(0, exp(-t0))

In [14]:
l_ZINB.diff(t0, 2)

-y*exp(g)*exp(t0)/(exp(g)*exp(t0) + 1) + y*exp(2*g)*exp(2*t0)/(exp(g)*exp(t0) + 1)**2 + (exp(g)*exp(t0) + 1)**(2*exp(-t0))*(exp(-t0)*log(exp(g)*exp(t0) + 1) - exp(g)/(exp(g)*exp(t0) + 1))**2/((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**2 - (exp(g)*exp(t0) + 1)**exp(-t0)*(exp(-t0)*log(exp(g)*exp(t0) + 1) - exp(g)/(exp(g)*exp(t0) + 1))**2/((exp(g)*exp(t0) + 1)**exp(-t0) - 1) + (exp(g)*exp(t0) + 1)**exp(-t0)*(-exp(-t0)*log(exp(g)*exp(t0) + 1) + exp(g)/(exp(g)*exp(t0) + 1) + exp(2*g)*exp(t0)/(exp(g)*exp(t0) + 1)**2)/((exp(g)*exp(t0) + 1)**exp(-t0) - 1) + exp(-t0)*polygamma(0, y + exp(-t0)) - exp(-t0)*polygamma(0, exp(-t0)) + exp(-2*t0)*polygamma(1, y + exp(-t0)) - exp(-2*t0)*polygamma(1, exp(-t0))

#### Mixed derivatives

Mixed derivatives involving $\eta$ are all 0 in the GAMLSS case. In the Extended GAM case, with $\eta=\theta_{1} + \exp(\theta_{2})\gamma$, we are interested in mixed derivatives involving $\gamma$ and $\theta_{i}, i=0,1,2$.

In [15]:
# for quasi-Newton
l_ZINB.diff(g, 1, t0, 1)

(-y*exp(t0) + y*exp(g)*exp(2*t0)/(exp(g)*exp(t0) + 1) - (exp(g)*exp(t0) + 1)**(2*exp(-t0))*(exp(-t0)*log(exp(g)*exp(t0) + 1) - exp(g)/(exp(g)*exp(t0) + 1))/((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**2 + (exp(g)*exp(t0) + 1)**exp(-t0)*(exp(-t0)*log(exp(g)*exp(t0) + 1) - exp(g)/(exp(g)*exp(t0) + 1))/((exp(g)*exp(t0) + 1)**exp(-t0) - 1) + (exp(g)*exp(t0) + 1)**exp(-t0)*exp(g)*exp(t0)/((exp(g)*exp(t0) + 1)*((exp(g)*exp(t0) + 1)**exp(-t0) - 1)))*exp(g)/(exp(g)*exp(t0) + 1)

In [16]:
# for quasi-Newton
l_ZINB.diff(g, 2, t0, 1)

(-y*exp(t0) + 3*y*exp(g)*exp(2*t0)/(exp(g)*exp(t0) + 1) - 2*y*exp(2*g)*exp(3*t0)/(exp(g)*exp(t0) + 1)**2 - (exp(g)*exp(t0) + 1)**(2*exp(-t0))*(exp(-t0)*log(exp(g)*exp(t0) + 1) - exp(g)/(exp(g)*exp(t0) + 1))/((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**2 + (exp(g)*exp(t0) + 1)**exp(-t0)*(exp(-t0)*log(exp(g)*exp(t0) + 1) - exp(g)/(exp(g)*exp(t0) + 1))/((exp(g)*exp(t0) + 1)**exp(-t0) - 1) + 2*(exp(g)*exp(t0) + 1)**(3*exp(-t0))*(exp(-t0)*log(exp(g)*exp(t0) + 1) - exp(g)/(exp(g)*exp(t0) + 1))*exp(g)/((exp(g)*exp(t0) + 1)*((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**3) + (exp(g)*exp(t0) + 1)**(2*exp(-t0))*(exp(-t0)*log(exp(g)*exp(t0) + 1) - exp(g)/(exp(g)*exp(t0) + 1))*exp(g)*exp(t0)/((exp(g)*exp(t0) + 1)*((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**2) - 3*(exp(g)*exp(t0) + 1)**(2*exp(-t0))*(exp(-t0)*log(exp(g)*exp(t0) + 1) - exp(g)/(exp(g)*exp(t0) + 1))*exp(g)/((exp(g)*exp(t0) + 1)*((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**2) - (exp(g)*exp(t0) + 1)**exp(-t0)*(exp(-t0)*log(exp(g)*exp(t0) + 1) - exp(g)/(exp(g)*exp(t0

In [17]:
# for full Newton
l_ZINB.diff(g, 1, t0, 2)

(-y*exp(t0) + 3*y*exp(g)*exp(2*t0)/(exp(g)*exp(t0) + 1) - 2*y*exp(2*g)*exp(3*t0)/(exp(g)*exp(t0) + 1)**2 - 2*(exp(g)*exp(t0) + 1)**(3*exp(-t0))*(exp(-t0)*log(exp(g)*exp(t0) + 1) - exp(g)/(exp(g)*exp(t0) + 1))**2/((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**3 + 3*(exp(g)*exp(t0) + 1)**(2*exp(-t0))*(exp(-t0)*log(exp(g)*exp(t0) + 1) - exp(g)/(exp(g)*exp(t0) + 1))**2/((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**2 - (exp(g)*exp(t0) + 1)**(2*exp(-t0))*(-exp(-t0)*log(exp(g)*exp(t0) + 1) + exp(g)/(exp(g)*exp(t0) + 1) + exp(2*g)*exp(t0)/(exp(g)*exp(t0) + 1)**2)/((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**2 - (exp(g)*exp(t0) + 1)**exp(-t0)*(exp(-t0)*log(exp(g)*exp(t0) + 1) - exp(g)/(exp(g)*exp(t0) + 1))**2/((exp(g)*exp(t0) + 1)**exp(-t0) - 1) + (exp(g)*exp(t0) + 1)**exp(-t0)*(-exp(-t0)*log(exp(g)*exp(t0) + 1) + exp(g)/(exp(g)*exp(t0) + 1) + exp(2*g)*exp(t0)/(exp(g)*exp(t0) + 1)**2)/((exp(g)*exp(t0) + 1)**exp(-t0) - 1) + 2*(exp(g)*exp(t0) + 1)**(2*exp(-t0))*(exp(-t0)*log(exp(g)*exp(t0) + 1) - exp(g)/(exp(g)*exp(t0) 

In [18]:
# for full Newton
l_ZINB.diff(g, 2, t0, 2)

(-y*exp(t0) + 7*y*exp(g)*exp(2*t0)/(exp(g)*exp(t0) + 1) - 12*y*exp(2*g)*exp(3*t0)/(exp(g)*exp(t0) + 1)**2 + 6*y*exp(3*g)*exp(4*t0)/(exp(g)*exp(t0) + 1)**3 - 2*(exp(g)*exp(t0) + 1)**(3*exp(-t0))*(exp(-t0)*log(exp(g)*exp(t0) + 1) - exp(g)/(exp(g)*exp(t0) + 1))**2/((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**3 + 3*(exp(g)*exp(t0) + 1)**(2*exp(-t0))*(exp(-t0)*log(exp(g)*exp(t0) + 1) - exp(g)/(exp(g)*exp(t0) + 1))**2/((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**2 - (exp(g)*exp(t0) + 1)**(2*exp(-t0))*(-exp(-t0)*log(exp(g)*exp(t0) + 1) + exp(g)/(exp(g)*exp(t0) + 1) + exp(2*g)*exp(t0)/(exp(g)*exp(t0) + 1)**2)/((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**2 - (exp(g)*exp(t0) + 1)**exp(-t0)*(exp(-t0)*log(exp(g)*exp(t0) + 1) - exp(g)/(exp(g)*exp(t0) + 1))**2/((exp(g)*exp(t0) + 1)**exp(-t0) - 1) + (exp(g)*exp(t0) + 1)**exp(-t0)*(-exp(-t0)*log(exp(g)*exp(t0) + 1) + exp(g)/(exp(g)*exp(t0) + 1) + exp(2*g)*exp(t0)/(exp(g)*exp(t0) + 1)**2)/((exp(g)*exp(t0) + 1)**exp(-t0) - 1) + 6*(exp(g)*exp(t0) + 1)**(4*exp(-t0))*(exp(-t0)

In [19]:
# for full Newton
l_ZINB.diff(g, 3, t0, 1)

(-y*exp(t0) + 7*y*exp(g)*exp(2*t0)/(exp(g)*exp(t0) + 1) - 12*y*exp(2*g)*exp(3*t0)/(exp(g)*exp(t0) + 1)**2 + 6*y*exp(3*g)*exp(4*t0)/(exp(g)*exp(t0) + 1)**3 - (exp(g)*exp(t0) + 1)**(2*exp(-t0))*(exp(-t0)*log(exp(g)*exp(t0) + 1) - exp(g)/(exp(g)*exp(t0) + 1))/((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**2 + (exp(g)*exp(t0) + 1)**exp(-t0)*(exp(-t0)*log(exp(g)*exp(t0) + 1) - exp(g)/(exp(g)*exp(t0) + 1))/((exp(g)*exp(t0) + 1)**exp(-t0) - 1) + 6*(exp(g)*exp(t0) + 1)**(3*exp(-t0))*(exp(-t0)*log(exp(g)*exp(t0) + 1) - exp(g)/(exp(g)*exp(t0) + 1))*exp(g)/((exp(g)*exp(t0) + 1)*((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**3) + 3*(exp(g)*exp(t0) + 1)**(2*exp(-t0))*(exp(-t0)*log(exp(g)*exp(t0) + 1) - exp(g)/(exp(g)*exp(t0) + 1))*exp(g)*exp(t0)/((exp(g)*exp(t0) + 1)*((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**2) - 9*(exp(g)*exp(t0) + 1)**(2*exp(-t0))*(exp(-t0)*log(exp(g)*exp(t0) + 1) - exp(g)/(exp(g)*exp(t0) + 1))*exp(g)/((exp(g)*exp(t0) + 1)*((exp(g)*exp(t0) + 1)**exp(-t0) - 1)**2) - 3*(exp(g)*exp(t0) + 1)**exp(-t0)*(ex

Further mixed derivatives can be found in [zinb](zinb.pdf)